In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import numpy as np
import sys
sys.path.append("./drive/MyDrive/287 Project")

In [3]:
import torchtext
from torchtext.data import get_tokenizer
from torchtext import data, datasets

In [4]:
from typing import *
import torch
from torch import nn, optim, Tensor
from torch.nn import CrossEntropyLoss
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import DataLoader, Dataset

In [5]:
import torch.nn.functional as F
from itertools import chain

In [6]:
!pip install transformers

     |████████████████████████████████| 3.3 MB 12.0 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
     |████████████████████████████████| 61 kB 684 kB/s 
     |████████████████████████████████| 596 kB 80.4 MB/s 
     |████████████████████████████████| 3.3 MB 60.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
from blog_data_reader import get_dataset
from example import get_hidden_size

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [10]:
# Load the Dataset
train, dev, test = get_dataset()

In [8]:
class BlogDataset(Dataset):
    def __init__(self, dataset):
      self.dataset = dataset
    def __len__(self) -> int:
        return len(self.dataset)
    def __getitem__(self, idx: int):
      example = self.dataset[idx]
      aux_labels = example.get_aux_labels()
      # Tokenized Text, Topic Label, Age of Author, Gender of Author, Length of Text
      return torch.tensor(example.get_tokenized()), torch.tensor(example.get_label()), torch.tensor(aux_labels[0]), torch.tensor(aux_labels[1]), torch.tensor(len(example.get_tokenized()))

In [11]:
train_ds = BlogDataset(train)
val_ds = BlogDataset(dev)
test_ds = BlogDataset(test)

In [12]:
def pad_collate_classifier(batch):
    (xx, yy, aa, bb, cc) = zip(*batch)
    xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
    yy_stack = torch.stack(yy, dim=0)
    aa_stack = torch.stack(aa, dim=0)
    bb_stack = torch.stack(bb, dim=0)
    cc_stack = torch.stack(cc, dim=0)
    return xx_pad, yy_stack, aa_stack, bb_stack, cc_stack

train_dl = DataLoader(train_ds, batch_size=256, shuffle=True, drop_last=True, collate_fn=pad_collate_classifier) 
val_dl = DataLoader(val_ds, batch_size=256, shuffle=False, drop_last=False, collate_fn=pad_collate_classifier)
test_dl = DataLoader(test_ds, batch_size=256, shuffle=False, drop_last=False, collate_fn=pad_collate_classifier) 

In [18]:
# Base LSTM Class
class EncoderLSTM(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, padding_idx: int):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.padding_idx = padding_idx
        
        self.dropout = nn.Dropout(0.2)
        self.embeddings = nn.Embedding(self.input_size, self.hidden_size, padding_idx=self.padding_idx)
        self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, num_layers=1, batch_first=True, bidirectional=True)

    def forward(self, input_seqs: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        input_lengths = []
        for review in input_seqs:
            count = 0
            for word in review:
                if word == self.padding_idx:
                    break
                count += 1
            input_lengths.append(count)

        embeddings = self.embeddings(input_seqs)
        embeddings =self.dropout(embeddings)
        
        inputs = pack_padded_sequence(embeddings, input_lengths, batch_first=True, enforce_sorted=False)
        packed_output, (ht, ct) = self.lstm(inputs)
        return (ht[0], ct[0], ht[1], ct[1])

In [19]:
# LSTM Embedding + Variational Encoder
class SentenceVAE(nn.Module):
  def __init__(self, input_size: int, embedding_size: int, hidden_size: int, padding_idx: int, dropout=0.0, num_layers=1, bidirectional=True):
        super().__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.padding_idx = padding_idx
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(dropout)
        # LSTM Embeddings
        self.encoder = EncoderLSTM(self.input_size, self.hidden_size, self.padding_idx)

        # Output mean and diagonal covariance from embeddings
        self.hidden2mean = nn.Linear(400, 128)
        self.hidden2logv = nn.Linear(400, 128)

  def encode(self, input_sequence):
      (ht0, ct0, ht1, ct1)= self.encoder(input_sequence)
      hidden = torch.cat((ht0, ct0, ht1, ct1), dim=1)

      z_loc = self.hidden2mean(hidden)
      z_scale = self.hidden2logv(hidden)
      return z_loc, z_scale
  
  def forward(self, input_sequence):
      mu, logvar = self.encode(input_sequence)
      return mu, logvar

In [20]:
# Base classifier network
class Classifier(torch.nn.Module):
  def __init__(self, dims=[2000,1024,512,20], dropout=0.0):
    super().__init__()
    self.latent_size = dims[0]
    self.output_size = dims[-1]

    self.classifier = nn.ModuleDict({
            f'layer_{i}':nn.Linear(dims[i],dims[i+1]) 
            for i in range(len(dims)-1)
        })
    self.dropout = nn.Dropout(p=dropout)

  def forward(self, hidden):
    hid = hidden
    for i,layer in self.classifier.items():
        hid = self.dropout(layer(hid))
    return hid

In [23]:
# Our entire network
class Disentangler(nn.Module):
  def __init__(self, num_target_labels, num_sensitive_labels):
    super().__init__()

    self.num_target_labels = num_target_labels
    self.num_sensitive_labels = num_sensitive_labels
    self.vocab_size = get_hidden_size()
    self.padding_idx = 0

    self.embedding_size = 400
    self.hidden_size = 100

    self.mu = None
    self.sigma = None

    self.encoder_layers = SentenceVAE(self.vocab_size, self.embedding_size, self.hidden_size, self.padding_idx, dropout=0.2)

    self.target_server = Classifier(dims=[64, 32, num_target_labels], dropout=0.2)
    self.target_adversary = Classifier(dims=[64, 32, num_sensitive_labels], dropout=0.2)

    self.sensitive_server = Classifier(dims=[64, 32, num_target_labels], dropout=0.2)
    self.sensitive_adversary = Classifier(dims=[64, 32, num_sensitive_labels], dropout=0.2)

    self.loss_fn = nn.CrossEntropyLoss()

  def encoder(self, x):
    # Return mean and covariance matrix from variational encoder
    return self.encoder_layers(x)
  
  def sample(self, z_loc, z_scale):
    # Sample from the outputted distribution (output from variational encoder)
    if self.training:
      std = torch.exp(0.5 * z_scale) 
      eps = torch.randn_like(std)
      return eps.mul_(std).add_(z_loc)
    else:
      return z_loc

  def forward_adversaries(self, x):
    # First forward step: Optimize the adveraries
    z_loc, z_scale = self.encoder(x)
    hidden = self.sample(z_loc, z_scale)

    server_hidden, adv_hidden = torch.split(hidden, 64, dim=1)

    target_adversary_logits = self.target_adversary(server_hidden)
    sensitive_server_logits = self.sensitive_server(adv_hidden)

    return target_adversary_logits, sensitive_server_logits
  
  def forward_all(self, x):
    # Forward step for entire network
    z_loc, z_scale = self.encoder(x)
    self.mu = z_loc
    self.sigma = z_scale

    hidden = self.sample(z_loc, z_scale)

    # Split Hidden State in half, one is target hidden and one is sensitive hidden
    server_hidden, adv_hidden = torch.split(hidden, 64, dim=1)

    target_server_logits = self.target_server(server_hidden)
    target_adversary_logits = self.target_adversary(server_hidden)

    sensitive_server_logits = self.sensitive_server(adv_hidden)
    sensitive_adversary_logits = self.sensitive_adversary(adv_hidden)

    return target_server_logits, target_adversary_logits, sensitive_server_logits, sensitive_adversary_logits
  
  def compute_loss_adversaries(self, target_adversary_logits, sensitive_server_logits, y_target, y_sensitive):
    # Compute the loss of the adversary networks
    target_adversary_loss = self.loss_fn(target_adversary_logits, y_sensitive)
    sensitive_server_loss = self.loss_fn(sensitive_server_logits, y_target)
    return train_params['target_adversary_weight'] * target_adversary_loss + train_params['sensitive_server_weight'] * sensitive_server_loss
  
  def compute_classifier_loss(self, target_server_logits, target_adversary_logits, sensitive_server_logits, sensitive_adversary_logits, y_target, y_sensitive):
    # Compute Target Server Loss, Combined Adversary Loss, and Sensitive Adversary Loss
    combined_adversary_loss = self.compute_loss_adversaries(target_adversary_logits, sensitive_server_logits, y_target, y_sensitive)
    target_server_loss = self.loss_fn(target_server_logits, y_target)
    sensitive_adversary_loss = self.loss_fn(sensitive_adversary_logits, y_sensitive)
    return target_server_loss, combined_adversary_loss, sensitive_adversary_loss

  def compute_total_loss(self, target_server_logits, target_adversary_logits, sensitive_server_logits, sensitive_adversary_logits, y_target, y_sensitive):
    # Function to compute total loss for second stage of the network
    target_server_loss, combined_adversary_loss, sensitive_adversary_loss = self.compute_classifier_loss(target_server_logits, target_adversary_logits, sensitive_server_logits, sensitive_adversary_logits, y_target, y_sensitive)
    # Computes KL Divergence from N(0, I)
    KL_term = -0.5 * torch.sum(1 + self.sigma - self.mu.pow(2) - self.sigma.exp())

    return  train_params['target_server_weight'] * target_server_loss + train_params['sensitive_adversary_weight'] * sensitive_adversary_loss - combined_adversary_loss + KL_term * train_params['kl_weight']

  def test_metrics(self, dl, adversary=False, sensitive=False):
    # Helper function to print out relevant metrics of our train loop
    self.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    loss_fn = nn.CrossEntropyLoss()
    
    for inputs, targets, age, gender, lengths in dl:
      if adversary:
        if not sensitive:
          targets = gender.cuda()
        else:
          targets = targets.cuda()
      else:
        if not sensitive:
          targets = targets.cuda()
        else:
          targets = gender.cuda()
      inputs = inputs.cuda()
      lengths = lengths.cuda()
      
      z_loc, z_scale = self.encoder(inputs)
      hidden = self.sample(z_loc, z_scale)
      server_hidden, adv_hidden = torch.split(hidden, 64, dim=1)

      if adversary:
        if not sensitive:
          model = self.target_adversary
        else:
          model = self.sensitive_server
      else:
        if not sensitive:
          model = self.target_server
        else:
          model = self.sensitive_adversary
      if sensitive:
        hidden = adv_hidden
      else:
        hidden = server_hidden
      logits = model(server_hidden)
      
      loss = loss_fn(logits, targets)
      sigmoid= torch.softmax(logits, dim=1)
      
      pred = torch.argmax(sigmoid, dim=1)
      correct += (pred.squeeze() == targets).sum()
      total += targets.shape[0]
      sum_loss += loss.item()*targets.shape[0]
    return sum_loss/total, correct.item()/total

  def train_all(self, train_params):
    # Adversary Parameters and Optimizer: We have two optimizers, one for each step of our training
    adversary_parameters = chain(self.target_adversary.parameters(), self.sensitive_server.parameters())
    adversary_optimizer = optim.Adam(adversary_parameters, lr=train_params['adv_lr'], weight_decay=train_params['adv_weight_decay'])

    # Parameters and Optimizer for the rest of our network
    disentangler_parameters = chain(self.encoder_layers.parameters(), self.target_server.parameters(), self.sensitive_adversary.parameters())
    disentangler_optimizer = optim.Adam(disentangler_parameters, lr=train_params['dis_lr'], weight_decay=train_params['dis_weight_decay'])

    epochs = train_params['epochs']
    for epoch in range(epochs):
      self.train()
      for i, (inputs, targets, age, gender, lengths) in enumerate(train_dl):
        sensitive = gender.cuda()
        inputs = inputs.cuda()
        lengths = lengths.cuda()
        targets = targets.cuda()

        #Optimize Adversary
        adversary_optimizer.zero_grad()
        target_adversary_logits, sensitive_server_logits = self.forward_adversaries(inputs)
        adversary_loss = self.compute_loss_adversaries(target_adversary_logits, sensitive_server_logits, targets, sensitive)

        adversary_loss.backward()
        adversary_optimizer.step()

        # Optimize Rest of our Network
        disentangler_optimizer.zero_grad()
        target_server_logits, target_adversary_logits, sensitive_server_logits, sensitive_adversary_logits = self.forward_all(inputs)
        loss = self.compute_total_loss(target_server_logits, target_adversary_logits, sensitive_server_logits, sensitive_adversary_logits, targets, sensitive)
        loss.backward()
        disentangler_optimizer.step()

      server_loss, server_acc = self.test_metrics(val_dl, adversary= False)
      adv_loss, adv_acc = self.test_metrics(val_dl, adversary= True)
      
      server_loss2, server_acc2 = self.test_metrics(val_dl, adversary= True, sensitive=True)
      adv_loss2, adv_acc2 = self.test_metrics(val_dl, adversary= False, sensitive=True)

      print(f"Target - Epoch {epoch}: SL {server_loss}, SA {server_acc}, AL {adv_loss}, AA {adv_acc}")
      print(f"Sensitive - Epoch {epoch}: SL {server_loss2}, SA {server_acc2}, AL {adv_loss2}, AA {adv_acc2}")
      print()

In [ ]:
d = Disentangler(10, 2).cuda()
# Hyperparameters
train_params = {
    'epochs': 100,
    'adv_lr': 1e-4,
    'adv_weight_decay': 1e-3,
    'dis_lr': 1e-3,
    'dis_weight_decay': 1e-4,
    'kl_weight': 1e-3,
    'target_server_weight': 1,
    'target_adversary_weight': 1,
    'sensitive_server_weight': 1,
    'sensitive_adversary_weight': 1
}
d.train_all(train_params)